# 因子值与股票明天的收益？

# 还记得我们的第一集的股价预测吗？
<!-- 我们用了很多的指标，但是我们并没有去验证这些指标的有效性，那么这一集我们就来验证一下这些指标的有效性。 -->





---
### 因子实战 第五集 
# 打造你的对冲基金：用Python实现高级对冲策略（3）
## 探秘因子统计学的解释性

### 🎬 @大导演哈罗德
### 🏛 香港中文大学 金融工程 本科
### 📈 即将前往美国金融工程硕士之路（已经获得录取）
### 🌐 [关注我的Bilibili，看所有人都能听得懂有收获的量化学习内容！](https://space.bilibili.com/629573485)

🌟🌟🌟 一起揭开量化的神秘面纱！#哈罗德的亮化频道🌟

---

读取数据

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

In [2]:
df_mkt = pd.read_excel('Monthly_Market_Value_and_Return.xlsx', index_col=0) 
df_mkt.reset_index(inplace=True)
df_mkt['stock'] = df_mkt['stock'].map(lambda x: str(x).zfill(6))
unique_dates = df_mkt['date'].unique()
unique_dates.sort()
unique_dates_nochange = unique_dates.copy()
unique_dates = [datetime.strptime(date, "%Y-%m") for date in unique_dates]
portfolio_returns = {f'portfolio_{i}': [] for i in range(1, 6)}

df = df_mkt

for date in unique_dates_nochange:
    df_date = df[df['date'] == date]
    if df_date.empty:
        continue
    
    thresholds = np.percentile(df_date['value_in_thousand'], [20, 40, 60, 80, 100])
    for i in range(5):
        if i == 0:
            portfolio = df_date[df_date['value_in_thousand'] <= thresholds[i]]
        else:
            portfolio = df_date[(df_date['value_in_thousand'] > thresholds[i-1]) & (df_date['value_in_thousand'] <= thresholds[i])]
        if portfolio.empty:
            portfolio_return = 0 
        else:
            if not np.isnan(portfolio['next_month_return'].mean()):
                portfolio_return = portfolio['next_month_return'].mean()

        portfolio_returns[f'portfolio_{i+1}'].append(portfolio_return)

hedge_returns = []
for i in range(len(portfolio_returns['portfolio_1'])):
    hedge_return = portfolio_returns['portfolio_1'][i] - portfolio_returns['portfolio_5'][i]
    hedge_returns.append(hedge_return)

average_returns = {portfolio: np.mean(returns) for portfolio, returns in portfolio_returns.items()}
average_hedge_return = np.mean(hedge_returns)

# 打印结果
print("Average Returns by Portfolio:")
for portfolio, average_return in average_returns.items():
    print(f"{portfolio}: {average_return}")

print(f"Average Hedge Return: {average_hedge_return}")

Average Returns by Portfolio:
portfolio_1: 0.03632338665770842
portfolio_2: 0.0209551217564462
portfolio_3: 0.01726534987882018
portfolio_4: 0.014766130366628655
portfolio_5: 0.012561872521888598
Average Hedge Return: 0.023761514135819815


In [3]:
hedge_returns

[-0.08470671268656715,
 0.03887293584905659,
 -0.06181304135338346,
 -0.0631031515151515,
 0.44565036015325665,
 0.055511378906250006,
 0.076745,
 -0.004596515267175572,
 0.046115068702290075,
 -0.03239070483133219,
 -0.14477963295880153,
 -0.0898985888888889,
 0.06386219413919417,
 0.17187434731115692,
 0.1655050845588235,
 0.022197353790613716,
 0.06864116725978646,
 -0.1442391094890511,
 0.10581406859205778,
 0.02646128776978418,
 -0.03362237234042553,
 -0.08557337234042553,
 0.15943493664975644,
 0.03087467368421054,
 0.04835371180555555,
 0.08164946917808219,
 0.049691647260273974,
 -0.1156161843003413,
 0.06389523469387756,
 -0.007008798657718113,
 0.09571214473684211,
 -0.004692326303989192,
 -0.07564928246753248,
 0.033560177419354825,
 0.07521826683369873,
 0.1149865145631068,
 0.020068660256410242,
 0.0686622134347432,
 0.06707708626198083,
 0.06279507667731629,
 0.04946440392950897,
 0.003032951768488723,
 -0.05047491693290737,
 0.0796321214057508,
 0.0016474190476190517,
 0

---

# 因子...?

你哈罗德不是说，可以根据一个因子确定一个股票的涨跌吗？

---

哈罗德双因子

In [4]:
SMB_monthly = pd.DataFrame(hedge_returns, unique_dates_nochange,columns=['SMB_monthly'])

pd.DataFrame(hedge_returns, unique_dates_nochange,columns=['SMB_monthly'])

,SMB_monthly
2005-12,-0.084707
2006-01,0.038873
2006-02,-0.061813
2006-03,-0.063103
2006-04,0.445650
...,...
2021-11,0.064914
2021-12,0.050056
2022-01,0.023067
2022-02,0.080904


In [5]:
df_mkt

,stock,date,value_in_thousand,monthly_return,next_month_return
0,000001,2005-12,11947347.99,0.051370,0.034202
1,000001,2006-01,12355970.65,0.034202,0.077165
2,000001,2006-02,13309423.50,0.077165,-0.068713
3,000001,2006-03,12394887.09,-0.068713,0.237049
4,000001,2006-04,15333078.53,0.237049,0.114213
...,...,...,...,...,...
419256,605599,2021-11,9636666.94,0.061697,0.086360
419257,605599,2021-12,10468889.19,0.086360,-0.116642
419258,605599,2022-01,9247778.04,-0.116642,0.105971
419259,605599,2022-02,10227778.07,0.105971,-0.170342


In [29]:
MKT = df_mkt.groupby('date')['monthly_return'].mean()
MKT_monthly = pd.DataFrame(MKT)
MKT_monthly

,monthly_return
date,
2005-12,0.027441
2006-01,0.088975
2006-02,0.032761
2006-03,0.011739
2006-04,0.088626
...,...
2021-11,0.081611
2021-12,0.048813
2022-01,-0.082702


现在我们有了每个月的 MKT factor 和 SMB factor 的值
### 对于任何一个股票，这个股票每个月的收益，我可以分解为 R = A1 * MKT factor + A2 * SMB factor + alpha

In [30]:
test_stock_code = '600000' #浦发银行
test_stock = df_mkt[df_mkt['stock'] == test_stock_code]
test_stock

,stock,date,value_in_thousand,monthly_return,next_month_return
208370,600000,2005-12,3.817125e+07,0.106697,0.155897
208371,600000,2006-01,4.412205e+07,0.155897,0.081633
208372,600000,2006-02,4.772385e+07,0.081633,-0.109106
208373,600000,2006-03,4.251690e+07,-0.109106,0.209055
208374,600000,2006-05,3.903255e+07,0.209055,-0.006018
...,...,...,...,...,...
208560,600000,2021-11,2.497869e+08,-0.048098,0.002350
208561,600000,2021-12,2.503740e+08,0.002350,-0.014068
208562,600000,2022-01,2.468517e+08,-0.014068,-0.002378
208563,600000,2022-02,2.462647e+08,-0.002378,-0.046484


In [41]:
X = pd.concat([SMB_monthly['SMB_monthly'], MKT_monthly['monthly_return']],axis= 1)
X

,SMB_monthly,monthly_return
2005-12,-0.084707,0.027441
2006-01,0.038873,0.088975
2006-02,-0.061813,0.032761
2006-03,-0.063103,0.011739
2006-04,0.445650,0.088626
...,...,...
2021-11,0.064914,0.081611
2021-12,0.050056,0.048813
2022-01,0.023067,-0.082702
2022-02,0.080904,0.049878


In [42]:
# regression on next_month_return on SMB
import statsmodels.api as sm
from statsmodels import regression

test_stock.fillna(0, inplace=True)

# X = SMB_monthly['SMB_monthly'],MKT_monthly['monthly_return']
y = test_stock[['date','next_month_return']].set_index('date')
# y_shift = y.shift(1).dropna()
# y = y_shift

data_for_stock = y.index.unique()
# y = y.loc[data_for_stock]
X = X.loc[data_for_stock]

X = sm.add_constant(X)
model = regression.linear_model.OLS(y, X).fit()
alpha = model.params[0]
beta1 = model.params[1]
beta2 = model.params[2]
print('alpha: ' + str(alpha))
print('beta SMB: ' + str(beta1))
print('beta MKT: ' + str(beta2))
print(model.summary())

alpha: 0.020873707410296646
beta SMB: -0.39489463362000604
beta MKT: 0.1579142085592274
                            OLS Regression Results                            
Dep. Variable:      next_month_return   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     5.808
Date:                Mon, 15 Jan 2024   Prob (F-statistic):            0.00356
Time:                        23:52:12   Log-Likelihood:                 170.73
No. Observations:                 195   AIC:                            -335.5
Df Residuals:                     192   BIC:                            -325.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

/var/folders/md/wt8g61857ld_20f_r6rn4nk40000gn/T/ipykernel_14659/1714119941.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_stock.fillna(0, inplace=True)
/var/folders/md/wt8g61857ld_20f_r6rn4nk40000gn/T/ipykernel_14659/1714119941.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0]
/var/folders/md/wt8g61857ld_20f_r6rn4nk40000gn/T/ipykernel_14659/1714119941.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[

还记得我们的shift吗？

In [43]:
# regression on next_month_return on SMB
import statsmodels.api as sm
from statsmodels import regression

test_stock.fillna(0, inplace=True)

# X = SMB_monthly['SMB_monthly'],MKT_monthly['monthly_return']
y = test_stock[['date','next_month_return']].set_index('date')
y_shift = y.shift(1).dropna()
y = y_shift

data_for_stock = y.index.unique()
# data_for_stock.sort()
# y = y.loc[data_for_stock]
X = X.loc[data_for_stock]

X = sm.add_constant(X)
model = regression.linear_model.OLS(y, X).fit()
alpha = model.params[0]
beta1 = model.params[1]
beta2 = model.params[2]
print('alpha: ' + str(alpha))
print('beta SMB: ' + str(beta1))
print('beta MKT: ' + str(beta2))
print(model.summary())

alpha: 0.008411295349334892
beta SMB: -0.16178903545541162
beta MKT: 0.5300906425337744
                            OLS Regression Results                            
Dep. Variable:      next_month_return   R-squared:                       0.238
Model:                            OLS   Adj. R-squared:                  0.230
Method:                 Least Squares   F-statistic:                     29.78
Date:                Mon, 15 Jan 2024   Prob (F-statistic):           5.54e-12
Time:                        23:52:29   Log-Likelihood:                 190.00
No. Observations:                 194   AIC:                            -374.0
Df Residuals:                     191   BIC:                            -364.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

/var/folders/md/wt8g61857ld_20f_r6rn4nk40000gn/T/ipykernel_14659/1988101145.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_stock.fillna(0, inplace=True)
/var/folders/md/wt8g61857ld_20f_r6rn4nk40000gn/T/ipykernel_14659/1988101145.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha = model.params[0]
/var/folders/md/wt8g61857ld_20f_r6rn4nk40000gn/T/ipykernel_14659/1988101145.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[

---

# 三因子，还有....

---

In [18]:
df_pb = pd.read_excel('Monthly_PB_and_Return.xlsx', index_col=0)
df_pb.reset_index(inplace=True)
df_pb['stock'] = df_pb['stock'].map(lambda x: str(x).zfill(6))
df_pb

,stock,name,date,return,PB,next_month_return
0,600000.SH,浦发银行,2005-12-30,10.6697,2.5770,15.5897
1,600000.SH,浦发银行,2006-01-25,15.5897,2.9788,8.1633
2,600000.SH,浦发银行,2006-02-28,8.1633,3.2219,-10.9106
3,600000.SH,浦发银行,2006-03-31,-10.9106,2.8704,0.0000
4,600000.SH,浦发银行,2006-04-28,0.0000,2.8704,20.9181
...,...,...,...,...,...,...
419985,003816.SZ,中国广核,2021-11-30,-0.6849,1.4650,7.9310
419986,003816.SZ,中国广核,2021-12-31,7.9310,1.5812,-7.9872
419987,003816.SZ,中国广核,2022-01-28,-7.9872,1.4549,2.0833
419988,003816.SZ,中国广核,2022-02-28,2.0833,1.4852,-7.1429


In [25]:
df = df_pb.copy()
df['book_to_market'] = 1 / df['PB']
df

,stock,name,date,return,PB,next_month_return,book_to_market
0,600000.SH,浦发银行,2005-12-30,10.6697,2.5770,15.5897,0.388048
1,600000.SH,浦发银行,2006-01-25,15.5897,2.9788,8.1633,0.335706
2,600000.SH,浦发银行,2006-02-28,8.1633,3.2219,-10.9106,0.310376
3,600000.SH,浦发银行,2006-03-31,-10.9106,2.8704,0.0000,0.348384
4,600000.SH,浦发银行,2006-04-28,0.0000,2.8704,20.9181,0.348384
...,...,...,...,...,...,...,...
419985,003816.SZ,中国广核,2021-11-30,-0.6849,1.4650,7.9310,0.682594
419986,003816.SZ,中国广核,2021-12-31,7.9310,1.5812,-7.9872,0.632431
419987,003816.SZ,中国广核,2022-01-28,-7.9872,1.4549,2.0833,0.687332
419988,003816.SZ,中国广核,2022-02-28,2.0833,1.4852,-7.1429,0.673310


# TODO: 利用Book to Market Ratio构建投资组合

# 如何在中国市场复现Fama-French三因子模型？

### 参考：liu+stambagh+yu+2019